In [1]:
from datetime import datetime, date, timedelta
from typing import Dict, List
import os
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.ext.declarative import declared_attr
import uuid
from sqlalchemy.dialects.postgresql import UUID


app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = "sqlite://"
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False

db = SQLAlchemy(app)


In [2]:
a = ''

b = '0.57'

c = float(b)
print(c)

0.57


In [2]:
class Business(db.Model):  # type: ignore
    """ Business parent model """
    __tablename__ = "business"

    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(24))
    created_on = db.Column(db.DateTime, default=datetime.utcnow)
    created_by = db.Column(db.Integer,  db.ForeignKey('user.id', name='fk_user_business_created_by'))
    employees = db.relationship('User', backref="employer", primaryjoin='User.employer_id==Business.id')

    
    
class User(db.Model):  # type: ignore
    """ User parent model """
    __tablename__ = "user"

    id = db.Column(db.Integer, primary_key=True)
    employer_id = db.Column(db.Integer, db.ForeignKey('business.id', name='fk_business_user_employer_id'))
    created_businesses = db.relationship('Business', backref='creator', order_by="desc(Business.created_on)", primaryjoin='Business.created_by==User.id', post_update=True)
    items = db.relationship('Item', backref="user", lazy='select')

    
    
class Item(db.Model):  # type: ignore
    """ User parent model """
    __tablename__ = "item"

    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id', name='fk_user_item_user_id'))
    
    

In [3]:
#db.reflect()
print("committing") #https://stackoverflow.com/questions/24289808/drop-all-freezes-in-flask-with-sqlalchemy
db.session.remove()

print("Dopping all")
db.drop_all()

print("Creating all")
db.create_all()

committing
Dopping all
Creating all


In [4]:
b1 = Business(name="b1")
b2 = Business(name="b2")

u1 = User()
u2 = User()

i1 = Item()
i2 = Item()

db.session.add(b1)
db.session.add(b2)
db.session.add(u1)
db.session.add(u2)
db.session.add(i1)
db.session.add(i2)

db.session.commit()

b1.employees.append(u1)
b1.employees.append(u2)
u1.created_businesses.append(b1)
u2.created_businesses.append(b2)
u1.items.append(i1)
u1.items.append(i2)
db.session.commit()

print(b1.employees)
print(u1.employer.name)
print(u1.created_businesses)
print(b2.created_by)
print(u1.items)


[<User 1>, <User 2>]
b1
[<Business 1>]
2
[<Item 1>, <Item 2>]
